In [13]:
from dotenv import load_dotenv

load_dotenv()
import nest_asyncio
import os

nest_asyncio.apply()
import json
import base64

In [14]:
# Load json from data/data.json

with open("data/data.json") as f:
    data = json.load(f)
print(data["script"])

# Constants:
img_urls = ["""https://cdn.discordapp.com/attachments/1212193794494042202/1223728760948523190/jas1.jpg?ex=661ae938&is=66087438&hm=a2661c174ebec9afba3265e3eb6bb86832f732f66d88195a5faa293a09839654&""", """https://cdn.discordapp.com/attachments/1221634008656642048/1223822939875704904/IMG_2084.jpg?ex=661b40ee&is=6608cbee&hm=ec453c6196ea2b6fc3bfa3e9fcb7f14a438f955f69198d937ea079da55a2afd6&"""]
voice_ids = ["jA3XNbtepbUtYOz5Q5bI", "UCHQN7CfyPSQTpOZoSvi"]
bearer = os.environ.get("DID_API_KEY")
encoded = base64.b64encode(bearer.encode("utf-8")).decode("utf-8")
eleven = os.environ.get("ELEVENLABS_API_KEY")

Have you ever wondered how your contacts app works so fast? 🤔 It uses a special trick called a trie! 🪄 Imagine a tree 🌳 but instead of leaves, it has arrays! 📊 Each array represents a letter. To find your friend's name, you just follow the path of letters until you reach the end. 😎 Whether you have 3 friends or 3 million, it always takes the same number of steps! ⏰ That's the magic of a trie - it's like having a super speedy dictionary 📖💨 right in your pocket! 🤳 Pretty cool, huh? 😄 #CS50 #computerscience #datastructures


In [15]:
import asyncio
import aiohttp

async def post_talk(script):
    url = "https://api.d-id.com/talks"
    headers = {
        'accept': 'application/json',
        'authorization': f'Basic {encoded}',
        'Content-Type': 'application/json',
        'x-api-key-external': json.dumps({"elevenlabs": eleven})
    }
    data = {
        "script": {
            "type": "text",
            "subtitles": "false",
            "provider": {
                "type": "elevenlabs",
                "voice_id": voice_ids[1],
                "voice_config": {"stability": 0.3, "similarity_boost": 0.9},
            },
            "model_id": "eleven_multilingual_v2",
            "input": script,
        },
        "config": {"fluent": "false", "pad_audio": "0.0"},
        "source_url": img_urls[1],
    }

    async with aiohttp.ClientSession() as session:
        async with session.post(url, headers=headers, json=data) as response:
            # Check if the request was successful
            if response.status == 201:
                # Process the response here
                response_data = await response.json()
                id = response_data["id"]

            else:
                # Handle request errors here
                print(f"Failed to post data, status code: {response.status}")
                print(await response.text())
                return ""

        # Loop to check the status
        while True:
            async with session.get(f"{url}/{id}", headers=headers) as status_response:
                if status_response.status == 200:
                    status_data = await status_response.json()
                    if status_data["status"] == "done":
                        result_url = status_data["result_url"]
                        break
                    else:
                        # Wait for some time before checking the status again
                        await asyncio.sleep(5)
                else:
                    print(f"Failed to get data, status code: {status_response.status}")
                    return ""

        # Download the video
        async with session.get(result_url) as video_response:
            if video_response.status == 200:
                mp4_data = (
                    await video_response.read()
                )  # This is the binary data of the MP4 file
                return mp4_data
            else:
                print(f"Failed to download video, status code: {video_response.status}")
                return ""

In [16]:
video = asyncio.run(post_talk(data["script"]))

In [ ]:
# Save video
with open("data/headshot.mp4", "wb") as f:
    f.write(video)